# Challenge

Another approach to identifying fraudulent transactions is to look for outliers in the data. Standard deviation or quartiles are often used to detect outliers. Using this starter notebook, code two Python functions:

* One that uses standard deviation to identify anomalies for any cardholder.

* Another that uses interquartile range to identify anomalies for any cardholder.

## Identifying Outliers using Standard Deviation

In [1]:
# Initial imports
import pandas as pd
import calendar
import hvplot.pandas
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os

# Load .env enviroment variables
load_dotenv()



True

In [2]:
# Create a connection to the database
database_url = os.getenv("database_url")
engine = create_engine(database_url)

# loading data for all card holders from the database
query = """

SELECT
    t.date AS date,
    ch.id AS card_holder_id, 
    cc.card AS credit_card_number,
    t.amount AS amount,
    t.id AS transaction_id, 
    m.id AS merchant_id,
    m.name AS merchant_name,
    mc.name AS merchant_category_name
FROM card_holder ch
JOIN credit_card cc ON ch.id = cc.cardholder_id
JOIN transaction t ON cc.card = t.card
LEFT JOIN merchant m ON t.id_merchant = m.id
LEFT JOIN merchant_category mc ON m.id_merchant_category = mc.id;

        """

# Create a DataFrame from the query result. HINT: Use pd.read_sql(query, engine)
all_card_holders = pd.read_sql(query, engine)
all_card_holders

# Clean Data and Prepare 
all_card_holders.isnull().count()
all_card_holders.dropna()
all_card_holders['date'] = pd.to_datetime(all_card_holders['date'])
all_card_holders['amount'] = all_card_holders['amount'].str.replace('[$,]', '', regex=True)
all_card_holders['amount'] = pd.to_numeric(all_card_holders['amount'], errors='coerce')
display(all_card_holders.head())
display(all_card_holders.tail())

,date,card_holder_id,credit_card_number,amount,transaction_id,merchant_id,merchant_name,merchant_category_name
0,2018-01-01 21:35:10,13,3561954487988605,6.22,222,69,Dominguez PLC,food truck
1,2018-01-01 21:43:12,13,5135837688671496,3.83,2045,85,Patton-Rivera,bar
2,2018-01-01 22:41:21,10,213193946980303,9.61,395,82,Day-Murray,food truck
3,2018-01-01 23:13:30,4,4263694062533017,19.03,3309,5,Miller-Blevins,pub
4,2018-01-01 23:15:10,18,4498002758300,2.95,567,64,"Cline, Myers and Strong",restaurant


,date,card_holder_id,credit_card_number,amount,transaction_id,merchant_id,merchant_name,merchant_category_name
3495,2018-12-31 01:24:15,15,4723783028106084756,4.84,1979,106,Carter-Blackwell,pub
3496,2018-12-31 03:33:28,10,4165305432349489280,3.26,2342,64,"Cline, Myers and Strong",restaurant
3497,2018-12-31 05:53:58,19,5361779664174555,10.73,948,19,Santos-Fitzgerald,pub
3498,2018-12-31 08:22:17,23,4188164051171486,11.87,1168,54,Berry-Lopez,restaurant
3499,2018-12-31 09:50:25,15,4723783028106084756,19.75,2476,16,"Bryant, Thomas and Collins",pub


In [3]:
# Write function that locates outliers using standard deviation

def find_outliers_std(all_card_holders, column_name, threshold=3):
    mean = all_card_holders[column_name].mean()
    std_dev = all_card_holders[column_name].std()
    
    # Identifying outliers
    outliers = all_card_holders[(all_card_holders[column_name] < mean - threshold * std_dev) | 
                                (all_card_holders[column_name] > mean + threshold * std_dev)]
    return outliers

outliers_df = find_outliers_std(all_card_holders, 'amount')



In [4]:
# Find anomalous transactions for 3 random card holders
random_card_holders = all_card_holders['card_holder_id'].sample(n=4, random_state=1).unique()

anomalous_transactions_list = []

for card_holder in random_card_holders:
    card_holder_data = all_card_holders[all_card_holders['card_holder_id'] == card_holder]
    anomalies = find_outliers_std(card_holder_data, 'amount')
    anomalous_transactions_list.append(anomalies)

anomalous_transactions_std = pd.concat(anomalous_transactions_list)

anomalous_transactions_std


,date,card_holder_id,credit_card_number,amount,transaction_id,merchant_id,merchant_name,merchant_category_name
487,2018-02-19 22:48:25,18,344119623920892,1839.0,3098,95,Baxter-Smith,restaurant
925,2018-04-03 03:23:37,18,344119623920892,1077.0,1359,100,Townsend-Anderson,restaurant
1508,2018-06-03 20:02:28,18,344119623920892,1814.0,3139,123,"Boone, Davis and Townsend",pub
2425,2018-09-10 22:49:41,18,344119623920892,1176.0,1431,72,Lopez-Kelly,restaurant
3095,2018-11-17 05:30:43,18,344119623920892,1769.0,3252,18,Romero-Jordan,food truck
3324,2018-12-13 12:09:58,18,344119623920892,1154.0,1326,8,Russell-Thomas,restaurant
212,2018-01-22 08:07:03,16,5570600642865857,1131.0,1442,144,"Walker, Deleon and Wolf",restaurant
457,2018-02-17 01:27:19,16,5570600642865857,1430.0,2699,71,Greene LLC,restaurant
625,2018-03-05 08:26:08,16,5570600642865857,1617.0,2451,4,Mccarty-Thomas,bar
1454,2018-05-29 02:55:08,16,5570600642865857,1203.0,1757,62,"Cooper, Carpenter and Jackson",food truck


## Identifying Outliers Using Interquartile Range

In [5]:
# Write a function that locates outliers using interquartile range
import pandas as pd

def find_outliers_iqr(all_card_holders, column_name):
    Q1 = all_card_holders[column_name].quantile(0.25)
    Q3 = all_card_holders[column_name].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = all_card_holders[(all_card_holders[column_name] < lower_bound) | (all_card_holders[column_name] > upper_bound)]
    return outliers

outliers_df = find_outliers_iqr(all_card_holders, 'amount')


In [6]:
# Find anomalous transactions for 3 random card holders
random_card_holders = all_card_holders['card_holder_id'].sample(n=4, random_state=1).unique()

anomalous_transactions_list = []

for card_holder in random_card_holders:
    card_holder_data = all_card_holders[all_card_holders['card_holder_id'] == card_holder]
    anomalies = find_outliers_iqr(card_holder_data, 'amount')
    anomalous_transactions_list.append(anomalies)

anomalous_transactions_iqr = pd.concat(anomalous_transactions_list)

display(anomalous_transactions_iqr)

,date,card_holder_id,credit_card_number,amount,transaction_id,merchant_id,merchant_name,merchant_category_name
53,2018-01-07 01:10:54,18,344119623920892,175.0,3457,12,"Bell, Gonzalez and Lowe",pub
67,2018-01-08 11:15:36,18,344119623920892,333.0,812,95,Baxter-Smith,restaurant
487,2018-02-19 22:48:25,18,344119623920892,1839.0,3098,95,Baxter-Smith,restaurant
925,2018-04-03 03:23:37,18,344119623920892,1077.0,1359,100,Townsend-Anderson,restaurant
1508,2018-06-03 20:02:28,18,344119623920892,1814.0,3139,123,"Boone, Davis and Townsend",pub
1763,2018-06-30 01:56:19,18,344119623920892,121.0,654,20,Kim-Lopez,coffee shop
1832,2018-07-06 16:12:08,18,344119623920892,117.0,560,62,"Cooper, Carpenter and Jackson",food truck
1956,2018-07-18 09:19:08,18,344119623920892,974.0,136,19,Santos-Fitzgerald,pub
2363,2018-09-02 11:20:42,18,344119623920892,458.0,2103,10,Herrera Group,restaurant
2425,2018-09-10 22:49:41,18,344119623920892,1176.0,1431,72,Lopez-Kelly,restaurant


In [7]:
print(f"Anomalies found: Fraudulent activities detected!")

Anomalies found: Fraudulent activities detected!
